In [7]:
import pickle
import numpy as np

# Load the fertilizer prediction model and related objects
with open("fertilizer_model.pkl", "rb") as f:
    fertilizer_data = pickle.load(f)
    fertilizer_model = fertilizer_data["model"]
    soil_encoder = fertilizer_data["soil_encoder"]
    crop_encoder = fertilizer_data["crop_encoder"]
    scaler = fertilizer_data["scaler"]
    fertilizer_encoder = fertilizer_data["fertilizer_encoder"]

# Load the crop recommendation model and label encoder
with open("qda_model.pkl", "rb") as model_file:
    qda_model = pickle.load(model_file)

with open("label_encoder.pkl", "rb") as encoder_file:
    label_encoder = pickle.load(encoder_file)



In [ ]:

# Crop-to-fertilizer mapping (already defined)
crop_fertilizer_mapping = {
    "Paddy": ["Rice"],
    "Pulses": ["Lentil", "Chickpea", "Kidney Beans"],
    "Millets": ["Millets", "Mungbean", "Blackgram"],
    "Oil seeds": ["Pomegranate", "Banana", "Mango", "Grapes"],
    "Ground Nuts": ["Coffee"]
}

# Crop recommendation function
def recommend_crop(temperature, humidity, pH, rainfall, nitrogen, phosphorus, potassium):
    try:
        # Create input features for crop recommendation
        input_features = pd.DataFrame({
            'N': [nitrogen],
            'P': [phosphorus],
            'K': [potassium],
            'temperature': [temperature],
            'humidity': [humidity],
            'ph': [pH],
            'rainfall': [rainfall]
        })

        # Standardize the input features (important for the model)
        input_scaled = scaler.transform(input_features)

        # Predict crop using QDA model
        crop_encoded = qda_model.predict(input_scaled)[0]
        
        # Decode the predicted crop label
        predicted_crop = label_encoder.inverse_transform([crop_encoded])[0]
        
        return predicted_crop
    except Exception as e:
        return f"Error in crop recommendation: {str(e)}"

# Fertilizer recommendation function
def recommend_fertilizer_for_crop(temperature, humidity, moisture, soil_type, crop_type, nitrogen, phosphorus, potassium):
    try:
        # Step 1: Recommend crop based on input features
        predicted_crop = recommend_crop(temperature, humidity, pH=6.5, rainfall=200, nitrogen=nitrogen, phosphorus=phosphorus, potassium=potassium)
        print(f"🌾 Recommended Crop: {predicted_crop}")
        
        # Step 2: Map the recommended crop to its corresponding fertilizers
        if predicted_crop in crop_fertilizer_mapping:
            fertilizers = crop_fertilizer_mapping[predicted_crop]
        else:
            fertilizers = ["Default Fertilizer"]  # If crop not in mapping, default fertilizer

        # Choose the first fertilizer from the list (you can modify this logic)
        recommended_fertilizer = fertilizers[0]
        
        # Step 3: Use the fertilizer and other input features for fertilizer prediction
        # Encode the inputs for fertilizer prediction model
        crop_encoded = crop_encoder.transform([predicted_crop])[0]
        soil_encoded = soil_encoder.transform([soil_type])[0]  # Example soil type input
        
        # Create feature array for fertilizer prediction
        input_features = np.array([[temperature, humidity, moisture, soil_encoded, crop_encoded, nitrogen, phosphorus, potassium]])
        
        # Standardize the input features for prediction
        input_scaled = scaler.transform(input_features)
        
        # Predict fertilizer using voting classifier model
        fertilizer_encoded = fertilizer_model.predict(input_scaled)[0]
        
        # Decode the predicted fertilizer
        predicted_fertilizer = fertilizer_encoder.inverse_transform([fertilizer_encoded])[0]
        
        return predicted_fertilizer

    except Exception as e:
        return f"Error in fertilizer recommendation: {str(e)}"

# Example usage for both crop and fertilizer recommendation
temperature = 28
humidity = 60
moisture = 40  # Example value
soil_type = "Loamy"  # Example soil type
crop_type = "Rice"  # This will be replaced with the crop recommended by the model
nitrogen = 20
phosphorus = 30
potassium = 10

# Get the fertilizer recommendation based on the crop type
predicted_fertilizer = recommend_fertilizer_for_crop(temperature, humidity, moisture, soil_type, crop_type, nitrogen, phosphorus, potassium)

print(f"🌾 Recommended Fertilizer: {predicted_fertilizer}")

🌾 Recommended Crop: Error in crop recommendation: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- K
- N
- P
- humidity
- ph
- ...
Feature names seen at fit time, yet now missing:
- Crop Type
- Humidity
- Moisture
- Nitrogen
- Phosphorous
- ...

🌾 Recommended Fertilizer: Error in fertilizer recommendation: y contains previously unseen labels: 'Error in crop recommendation: The feature names should match those that were passed during fit.\nFeature names unseen at fit time:\n- K\n- N\n- P\n- humidity\n- ph\n- ...\nFeature names seen at fit time, yet now missing:\n- Crop Type\n- Humidity\n- Moisture\n- Nitrogen\n- Phosphorous\n- ...\n'


In [8]:
def recommend_crop(temperature, humidity, moisture, soil_type, nitrogen, phosphorous, potassium):
    try:
        # Encode categorical inputs
        soil_encoded = soil_encoder.transform([soil_type])[0]
        
        # Create feature array and standardize
        input_features = np.array([[temperature, humidity, moisture, soil_encoded, nitrogen, phosphorous, potassium]])
        input_scaled = scaler.transform(input_features)
        
        # Predict crop using QDA model
        crop_encoded = qda_model.predict(input_scaled)[0]
        predicted_crop = label_encoder.inverse_transform([crop_encoded])[0]
        # print("🌾Crop Recommendation: ",predicted_crop)
        return predicted_crop
    
    except Exception as e:
        return f"Error in crop recommendation: {str(e)}"


In [9]:
def recommend_fertilizer_for_crop(temperature, humidity, moisture, soil_type, nitrogen, phosphorous, potassium):
    try:
        # Step 1: Recommend crop based on the given features
        predicted_crop = recommend_crop(temperature, humidity, moisture, soil_type, nitrogen, phosphorous, potassium)
        print(f"🌾 Recommended Crop: {predicted_crop}")
        
        # Step 2: Use the recommended crop as input for fertilizer prediction
        soil_encoded = soil_encoder.transform([soil_type])[0]
        crop_encoded = crop_encoder.transform([predicted_crop])[0]
        
        # Create feature array and standardize
        input_features = np.array([[temperature, humidity, moisture, soil_encoded, crop_encoded, nitrogen, phosphorous, potassium]])
        input_scaled = scaler.transform(input_features)
        
        # Predict fertilizer using the fertilizer prediction model
        fertilizer_encoded = fertilizer_model.predict(input_scaled)[0]
        predicted_fertilizer = fertilizer_encoder.inverse_transform([fertilizer_encoded])[0]
        # print(f"🌱 Recommended Fertilizer: {predicted_fertilizer}")
        return predicted_fertilizer
    
    except Exception as e:
        return f"Error in fertilizer recommendation: {str(e)}"


In [ ]:
# Example features
temperature = 28
humidity = 60
moisture = 40
soil_type = "Loamy"
nitrogen = 20
phosphorous = 30
potassium = 10
crop_type = 
# Get the recommended fertilizer based on the crop recommendation
predicted_fertilizer = recommend_fertilizer_for_crop(temperature, humidity, moisture, soil_type, nitrogen, phosphorous, potassium)
print(f"🌾 Recommended Fertilizer: {predicted_fertilizer}")


🌾 Recommended Crop: Error in crop recommendation: X has 7 features, but StandardScaler is expecting 8 features as input.
🌾 Recommended Fertilizer: Error in fertilizer recommendation: y contains previously unseen labels: 'Error in crop recommendation: X has 7 features, but StandardScaler is expecting 8 features as input.'


d:\major project ideas\Project implementation\mpenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [1]:
def get_modified_crop_type(crop_type: str) -> str:
    """
    Returns the modified crop type based on the crop_fertilizer_mapping.
    If no mapping is found, returns the original crop type.
    """
    crop_fertilizer_mapping = {
        "Paddy": ["Rice"],
        "Pulses": ["Lentil", "Chickpea", "Kidney Beans"],
        "Millets": ["Millets", "Mungbean", "Blackgram"],
        "Oil seeds": ["Pomegranate", "Banana", "Mango", "Grapes"],
        "Ground Nuts": ["Coffee"]
    }

    for main_crop, crops in crop_fertilizer_mapping.items():
        if crop_type in crops:
            return main_crop
    return crop_type  

print(get_modified_crop_type("Rice"))  # Output: Paddy

Paddy
